In [ ]:
import argparse

from keras import utils
from keras import optimizers
from keras.datasets import mnist
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential

# Polyaxon
from polyaxon_client.tracking import Experiment

In [ ]:
OPTIMIZERS = {
    'adam': optimizers.Adam,
    'rmsprop': optimizers.RMSprop,
    'sgd': optimizers.SGD,
}

In [ ]:
def transform_data(x_train, y_train, x_test, y_test):
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_train = x_train.astype('float32') / 255

    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    x_test = x_test.astype('float32') / 255

    y_train = utils.to_categorical(y_train, num_classes=10)
    y_test = utils.to_categorical(y_test, num_classes=10)

    return x_train, y_train, x_test, y_test


def train(conv1_size, conv2_size, dropout, hidden1_size, optimizer, log_learning_rate, epochs):
    model = Sequential()
    model.add(Conv2D(filters=conv1_size,
                     kernel_size=(3, 3),
                     activation='relu',
                     input_shape=x_train.shape[1:]))
    model.add(Conv2D(filters=conv2_size,
                     kernel_size=(3, 3),
                     activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))
    model.add(Flatten())
    model.add(Dense(hidden1_size, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=OPTIMIZERS[optimizer](lr=10 ** log_learning_rate),
        loss='categorical_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train, epochs=epochs, batch_size=100)

    return model.evaluate(x_test, y_test)[1]

In [ ]:
conv1_size_param=32
conv2_size_param=64
dropout_param=0.8
hidden1_size_param=500
optimizer_param='adam'
log_learning_rate_param=-3
epochs_param=1

In [ ]:
# Polyaxon
experiment = Experiment(project='mnist')
experiment.create(tags=['keras'])
experiment.log_params(conv1_size=conv1_size_param,
                      conv2_size=conv2_size_param,
                      dropout=dropout_param,
                      hidden1_size=hidden1_size_param,
                      optimizer=optimizer_param,
                      log_learning_rate=log_learning_rate_param,
                      epochs=epochs_param)

(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Polyaxon
experiment.log_data_ref(data=x_train, data_name='x_train')
experiment.log_data_ref(data=y_train, data_name='y_train')
experiment.log_data_ref(data=x_test, data_name='x_test')
experiment.log_data_ref(data=y_test, data_name='y_test')

x_train, y_train, x_test, y_test = transform_data(x_train, y_train, x_test, y_test)
accuracy = train(conv1_size=conv1_size_param,
                  conv2_size=conv2_size_param,
                  dropout=dropout_param,
                  hidden1_size=hidden1_size_param,
                  optimizer=optimizer_param,
                  log_learning_rate=log_learning_rate_param,
                  epochs=epochs_param)

# Polyaxon
experiment.log_metrics(accuracy=accuracy)